In [1]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml

In [12]:
class BayesClassifier:
    def __init__(self):
        self.C = {}
        self.mu = {}
        
    def train(self, DTR, LTR):

        self.mu, self.C = ml.MU_Cov_calculator(DTR, LTR)
        for i in numpy.unique(LTR):
            self.C[i] *= numpy.eye(self.C[i].shape[0])

    def test(self, DTE, LTE):

        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            S[i, :] =ml.GAU_logpdf_ND(DTE, self.mu[i], self.C[i])  + numpy.log(1/2)

        Sp = scipy.special.logsumexp(S, axis=0)

        for x, p in zip(S.T, Sp):
            tmp = x - p
            predicted.append(numpy.argmax(tmp))

        predicted = numpy.array(predicted)
       
        True_prediction = numpy.array([predicted == LTE])
        error = 1 - (numpy.count_nonzero(True_prediction) / True_prediction.size)
        print("Bayes Classifier error:", error)


In [3]:
Data, label = ml.loadFile('../Train.txt')
(DTR, LTR), (DTE,LTE) = ml.split_db_2to1(Data, label, seed=42)

In [13]:
G=BayesClassifier()
G.train(DTR,LTR)
G.test(DTE,LTE)

Bayes Classifier error: 0.06315082297615049


In [14]:
Gn=BayesClassifier()
Gn.train(ml.z_normalization(DTR), LTR)
Gn.test(DTE, LTE)
Gn.test(ml.z_normalization(DTE), LTE)

Bayes Classifier error: 0.41182398387638564
Bayes Classifier error: 0.06348673160900231


In [15]:
GG=BayesClassifier()
GG.train(ml.gaussianize(DTR), LTR)
GG.test(DTE, LTE)
GG.test(ml.gaussianize(DTE), LTE)

Bayes Classifier error: 0.0923748740342627
Bayes Classifier error: 0.05844810211622442


In [24]:
class BayesClassifier_mod1:
    def __init__(self):
        self.C = {}
        self.mu = {}
        
    def train(self, DTR, LTR):

        self.mu, self.C = ml.MU_Cov_calculator(DTR, LTR)
        for i in numpy.unique(LTR):
            self.C[i] *= numpy.eye(self.C[i].shape[0])

    def test(self, DTE, LTE, app):
        
        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        ll=numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            ll[i, :]=ml.GAU_logpdf_ND(DTE, self.mu[i], self.C[i])
            S[i, :] =ll[i, :]  + numpy.log(1/2)

        Sp = scipy.special.logsumexp(S, axis=0)

        for x, p in zip(S.T, Sp):
            tmp = x - p
            predicted.append(numpy.argmax(tmp))

        predicted = numpy.array(predicted)
       
        app_bayes_risk=ml.compute_Bayes_risk(ml.compute_confusion_matrix(predicted, LTE), app)
        DCF = ml.compute_norm_Bayes(app_bayes_risk, app)
        
        llr = numpy.array(ll[1, :]-ll[0, :])
        minDCF= ml.compute_min_DCF(llr, app, LTE)

        True_prediction = numpy.array([predicted == LTE])
        error = 1 - (numpy.count_nonzero(True_prediction) / True_prediction.size)
      
        print("\-/ \-/ \-/ \-/ \-/ ")
        print("Tied Covariance Gaussian Classifier error:", error)
        print(app,"DCF:", DCF, "minDCF:", minDCF)
        print("/-\ /-\ /-\ /-\ /-\ ")

In [25]:
G=BayesClassifier_mod1()
G.train(DTR,LTR)
G.test(DTE,LTE, [1/2,1,1])

\-/ \-/ \-/ \-/ \-/ 
Tied Covariance Gaussian Classifier error: 0.06315082297615049
[0.5, 1, 1] DCF: 0.1936962519345939 minDCF: 0.19141107597066145
/-\ /-\ /-\ /-\ /-\ 
